In [50]:
import torch
import pandas as pd
from pathlib import Path
from typing import Dict
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead

In [51]:
!python prepare_messages.py --tg-history-path "data/result.json" --output-path "data/data.csv"

In [52]:
DATA_PATH = "data/data.csv"

data = load_dataset("csv", data_files=DATA_PATH, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/ninachely/.cache/huggingface/datasets/csv/default-d515fd3fd78f0310/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [53]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 121829
})


In [54]:
data = data.filter(lambda example: example["context_1"] != None)
print(data)

Filter:   0%|          | 0/121829 [00:00<?, ? examples/s]

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})


In [55]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})


In [56]:
data[0]

{'context_3': None,
 'context_2': None,
 'context_1': 'норм цвет)',
 'response': 'Грустно видеть Диану с ником куратора\nто есть ты больше не наш куратор?(('}

In [57]:
data = data.train_test_split(test_size=0.2, shuffle=True)

In [58]:
data['train'][0:1]

{'context_3': ['Много чего на самом деле)\nНо меня именно от microsoft штуки интересуют, остальное от физтеховской есть (она гугловская)'],
 'context_2': ['Может быть будут работать студенческие подписки'],
 'context_1': ['Jetbrains же одной кнопкой продлевается\nБез всяких подтверждений статуса студента'],
 'response': ['Никто не в курсе, когда там приказы-то вообще появятся по коммерции?']}

In [59]:
FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'

CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']
SEP = ' '

def convert_to_dialog(sample: Dict[str, str]) -> Dict[str, str]:
    """
        Convert sample row to dialogs str format
    """
    result_dict = dict()
    dialog = ""
    for i in range(len(CONTEXT_COLS)):
        key = CONTEXT_COLS[i]
        if key in sample:
            speaker_token = FIRST_SPEAKER_TOKEN if i % 2 == 0 else SECOND_SPEAKER_TOKEN
            dialog += speaker_token + SEP + sample[key] + SEP
    
    response_key = RESPONSE_COL[0]
    if response_key in sample:
        dialog += SECOND_SPEAKER_TOKEN + SEP + sample[response_key]

    result_dict['text'] = dialog

    return result_dict

In [60]:
convert_to_dialog(
    {
        'context_3': 'привет',
        'context_2': 'привет!',
        'context_1': 'как дела?',
        'response': 'супер)'
    }
)

{'text': '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет! @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

In [61]:
convert_to_dialog(
    {
        'context_1': 'как дела?',
        'response': 'супер)'
    }
)

{'text': '@@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

In [63]:
assert convert_to_dialog(
    {
        'context_3': 'привет',
        'context_2': 'привет!',
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ привет @@ВТОРОЙ@@ привет! @@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

assert convert_to_dialog(
    {
        'context_1': 'как дела?',
        'response': 'супер)'
    }
) == {'text': '@@ПЕРВЫЙ@@ как дела? @@ВТОРОЙ@@ супер)'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')